In [690]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, GradientBoostingClassifier, GradientBoostingRegressor

In [701]:
df_train = pd.read_csv('./data/train_pre.csv').drop_duplicates()
df_test = pd.read_csv('./data/test_pre.csv').drop_duplicates()
drivers = pd.read_csv('./data/driver_mapping.csv')
circuit = pd.read_csv('./data/circuit_mapping.csv')

df_train.drop(columns=['results_points'], inplace=True)
df_test.drop(columns=['results_points'], inplace=True)

# drivers.drop(columns=['Unnamed: 0'], inplace=True)

In [692]:
def get_score(pred, actual, margin):
    if(np.argmin(pred) == np.argmin(actual)):
        return 1
    return 0

#### RF CLASSIFIER

In [693]:
def RF_classifier(X_train, Y_train):
    model = RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=100)
    model.fit(X_train, Y_train)

    return model

In [694]:
def get_RF_classifier_Score():
    X_train = df_train
    Y_train = df_train['results_positionOrder']
    model = RF_classifier(X_train.drop(columns = ['results_positionOrder']), Y_train, )
    race_nos = []
    predicted_winners = []
    actual_winners = []
    predicted_driver_ids = []
    actual_driver_ids = []
    predicted_winner_name = []
    actual_winner_name = []
    circuit_names = []

    score = 0
    for i in df_test['raceId'].unique():
        X_test = df_test[df_test['raceId'] == i]
        Y_test = df_test[df_test['raceId'] == i]['results_positionOrder']

        prediction = model.predict(X_test.drop(columns = ['results_positionOrder']))        
        actual = Y_test.to_numpy()        
        
        predicted_winner = np.argmin(prediction)
        actual_winner = np.argmin(actual)        
                
        race_nos.append(str(i))
        predicted_winners.append(str(predicted_winner))
        actual_winners.append(str(actual_winner))
        predicted_driver_ids.append(X_test['driverId'].iloc[predicted_winner])
        actual_driver_ids.append(X_test['driverId'].iloc[actual_winner])

        
        predicted_winner_name.append(drivers[np.isclose(drivers['driverId'], X_test['driverId'].iloc[predicted_winner])].driverRef.values[0])
        actual_winner_name.append(drivers[np.isclose(drivers['driverId'], X_test['driverId'].iloc[actual_winner])].driverRef.values[0])

        circuit_no = X_test['circuitId'].iloc[0]
        # print(circuit_no)
        circuit_names.append(circuit[np.isclose(circuit['circuitId'], circuit_no)].circuit_id.values[0])
        

        # circuit_ids.append(X_test['circuitId'].iloc[i])
        score += get_score(prediction, actual, 0)

    print("Score: " + str(score/len(df_test['raceId'].unique())))
    d = {'Predicted_winner': predicted_winners, 'Predicted Driver' : predicted_winner_name, 'Actual_winner': actual_winners, 'Actual Driver': actual_winner_name, 'Circuit': circuit_names}
    return(pd.DataFrame(d, index=race_nos))

In [695]:
print('2020')
get_RF_classifier_Score()

2020
Score: 0.7647058823529411


,Predicted_winner,Predicted Driver,Actual_winner,Actual Driver,Circuit
1045,0,hamilton,0,hamilton,bahrain
1036,0,hamilton,0,hamilton,catalunya
1044,0,hamilton,0,hamilton,istanbul
1034,0,hamilton,0,hamilton,silverstone
1035,0,hamilton,10,max_verstappen,silverstone
1041,0,hamilton,0,hamilton,nurburgring
1033,0,hamilton,0,hamilton,hungaroring
1037,0,hamilton,0,hamilton,spa
1038,0,hamilton,16,gasly,monza
1047,0,hamilton,9,max_verstappen,yas_marina
